In [ ]:
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/train_preprocessed.csv')

In [ ]:
val_df = pd.read_csv('/content/drive/MyDrive/College/Semester 5/NLP Project/NLP_Project-main/NLP_Project-main/Data/PreprocessedData/val_preprocessed.csv')

In [ ]:
train_X = train_df['preprocessed_text'].to_list()
train_Y = (train_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [ ]:
val_X = val_df['preprocessed_text'].to_list()
val_Y = (val_df['label'].replace('OFF',1)).replace('NOT', 0).to_list()

In [ ]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_X)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=128,
        mask_zero=True),
   tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 128)         1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,333,633
Trainable params: 1,333,633
Non-trainable params: 0
______________________________________________

In [ ]:
history = model.fit(x = train_X, y = train_Y, epochs=10,
                    validation_data=(val_X, val_Y),
                    validation_steps=100)

Epoch 1/10
330/331 [============================>.] - ETA: 0s - loss: 0.6489 - accuracy: 0.6714

331/331 [==============================] - 27s 67ms/step - loss: 0.6489 - accuracy: 0.6710 - val_loss: 0.6236 - val_accuracy: 0.6545
Epoch 2/10
331/331 [==============================] - 18s 54ms/step - loss: 0.5811 - accuracy: 0.6721
Epoch 3/10
331/331 [==============================] - 18s 54ms/step - loss: 0.4521 - accuracy: 0.7772
Epoch 4/10
331/331 [==============================] - 18s 54ms/step - loss: 0.3377 - accuracy: 0.8563
Epoch 5/10
331/331 [==============================] - 18s 54ms/step - loss: 0.2716 - accuracy: 0.8905
Epoch 6/10
331/331 [==============================] - 18s 54ms/step - loss: 0.2240 - accuracy: 0.9111
Epoch 7/10
331/331 [==============================] - 18s 54ms/step - loss: 0.1877 - accuracy: 0.9281
Epoch 8/10
331/331 [==============================] - 18s 54ms/step - loss: 0.1674 - accuracy: 0.9381
Epoch 9/10
331/331 [==============================] - 18s 54ms/step - loss: 0.1385 - accuracy: 0.9508
Epoch 10/10
331/331 [==============================

In [ ]:
train_pred = model.predict(train_X)
val_pred = model.predict(val_X)

83/83 [==============================] - 2s 13ms/step


In [ ]:
train_pred = np.where(train_pred > 0.5, 1, 0)
val_pred = np.where(val_pred > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

def computeAllScores(y_pred_train, y_pred_dev, train_labels, dev_labels):
    print("Accuracy Train: ", accuracy_score(train_labels, y_pred_train))
    print("Accuracy Dev: ", accuracy_score(dev_labels, y_pred_dev))
    print("Weighted F1 Train: ", f1_score(train_labels, y_pred_train, average='weighted'))
    print("Weighted F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='weighted'))
    print("Macro F1 Train: ", f1_score(train_labels, y_pred_train, average='macro'))
    print("Macro F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='macro'))
    print("Micro F1 Train: ", f1_score(train_labels, y_pred_train, average='micro'))
    print("Micro F1 Dev: ", f1_score(dev_labels, y_pred_dev, average='micro'))
    print("Weighted Recall Train: ", recall_score(train_labels, y_pred_train, average='weighted'))
    print("Weighted Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='weighted'))
    print("Macro Recall Train: ", recall_score(train_labels, y_pred_train, average='macro'))
    print("Macro Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='macro'))
    print("Micro Recall Train: ", recall_score(train_labels, y_pred_train, average='micro'))
    print("Micro Recall Dev: ", recall_score(dev_labels, y_pred_dev, average='micro'))
    # Confusion Matrix
    print("Confusion Matrix Train: ")
    print(confusion_matrix(train_labels, y_pred_train))
    print("Confusion Matrix Dev: ")
    print(confusion_matrix(dev_labels, y_pred_dev))

In [ ]:
computeAllScores(train_pred, val_pred, train_Y, val_Y)

Accuracy Train:  0.9634629909365559
Accuracy Dev:  0.7345166163141994
Weighted F1 Train:  0.9631101417211528
Weighted F1 Dev:  0.724120565231274
Macro F1 Train:  0.9578508007409474
Macro F1 Dev:  0.6859861715728305
Micro F1 Train:  0.9634629909365559
Micro F1 Dev:  0.7345166163141994
Weighted Recall Train:  0.9634629909365559
Weighted Recall Dev:  0.7345166163141994
Macro Recall Train:  0.9497408843181869
Macro Recall Dev:  0.677750450118087
Micro Recall Train:  0.9634629909365559
Micro Recall Dev:  0.7345166163141994
Confusion Matrix Train: 
[[7035   72]
 [ 315 3170]]
Confusion Matrix Dev: 
[[1493  240]
 [ 463  452]]
